<a href="https://colab.research.google.com/github/tunakan12/saikikannsu/blob/main/cDA25_10_%E6%BC%94%E7%BF%92%E8%AA%B2%E9%A1%8C2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#（１）共通ツール：データ生成，配列の出力などの関数群

##(1a)ヘッダーファイル

構造体：counter
  - compare: 比較回数
  - swap: 交換回数
  - move: 移動回数

準備した関数
- 基本関数
  - swap：指定した２データの交換
  - copyarray：配列データの複写
  - printarray：配列データの出力
- データ読み込み関数
  - allocate_member：新しいメンバを割り当てる
  - read_members_from_file：ファイルからメンバを読み込む
  - init_members_from_file：新しいメンバを読み込む
  - add_members_from_file：追加のメンバを読み込む
- 比較関数
  - compare_by_yomi：名前(読み)(yomi)で比較
  - compare_by_age：年齢(age)で比較
  - compare_by_height：身長(height)で比較
  - compare_by_score：強さ(score)で比較
  - compare_by_group：所属(group)で比較
- ソート関数
  - bubble_sort：バブルソート
  - bubble2_sort：改良バブルソート
  - shaker_sort：シェーカーソート
  - select_sort：選択ソート
  - insert_sort：挿入ソート
  - quick_sort：クイックソート

In [ ]:
%%file main.h
#define MAX_NAME  (12*3)
#define MAX_ATTR  (3*3)
// Member構造体の定義
typedef struct member {
    char  name[MAX_NAME+1];  // 名前(漢字表記)
    char  yomi[MAX_NAME+1];  // 読み(カタカナ表記)
    char  group[MAX_NAME+1]; // 所属
    char  attr[MAX_ATTR+1];  // 属性
    int   age;               // 年齢
    float height;            // 身長
    int   score;             // 強さ
} Member;
// 比較関数型へのポインタ
typedef int (*CmpFunc)(const Member *, const Member *, int);
// カウンタ型
typedef struct counter{
    int compare;
    int swap;
    int move;
} Counter;

// tool functions
Member *allocate_member(const char *name, const char *yomi, const char *group, const char *attr, int age, float height, int score);
Member **init_members_from_file(const char *filename, int *ret_num);
Member **add_members_from_file(const char *filename, int *ret_num, Member **member_array);
void swap(Member *data[], int idx1, int idx2);
void copyarray(int n, Member *data1[], Member *data2[]);
void printarray(int n, Member *data[]);

// counter functions
extern Counter ctr;
void print_counter(void);
void reset_counter(void);

// compare functions
int compare_by_yomi(const Member *data1, const Member *data2, int descending);
int compare_by_age(const Member *data1, const Member *data2, int descending);
int compare_by_height(const Member *data1, const Member *data2, int descending);
int compare_by_score(const Member *data1, const Member *data2, int descending);
int compare_by_group(const Member *data1, const Member *data2, int descending);

// sort functions
void launch_sort(int n, Member *data[], void (*sort_func)(int, Member**, CmpFunc, int), CmpFunc cmp, int descending, const char *sort_name, const char *cmp_name);
#define LAUNCH_SORT(n, data, sort_func, cmp, descending)  launch_sort(n, data, sort_func, cmp, descending, #sort_func, #cmp)
void bubble_sort(int n, Member *data[], CmpFunc cmp, int descending);
void bubble2_sort(int n, Member *data[], CmpFunc cmp, int descending);
void shaker_sort(int n, Member *data[], CmpFunc cmp, int descending);
void select_sort(int n, Member *data[], CmpFunc cmp, int descending);
void insert_sort(int n, Member *data[], CmpFunc cmp, int descending);
void quick_sort(int n, Member *data[], CmpFunc cmp, int descending);

Writing main.h


##(1b)データ生成，配列の出力，コピー

基本関数
- swap
 - 処理：指定した２データの交換
 - 入力：data[](Member*)：配列データ
 - 入力：idx1, 1dx2(int)：交換データのインデックス
- copyarray
 - 処理：配列データの複写
 - 入力：n(int)：データサイズ，data1[](Member*)：入力元配列データ
 - 出力：data2[](int)：出力先配列データ
- printarray
 - 処理：配列データの出力
 - 入力：n(int)：データサイズ，data[](Member*)：配列データ
 - 出力：出力先への配列データの出力

カウンタ操作
- print_counter
 - 処理：カウンタを表示(lap, all)
- reset_counter
 - 処理：現在のカウンタをトータルカウンタに足しこみ、リセット

メンバの割り当てと読み込み
- allocate_member
 - 処理：新しいメンバ用の構造体を確保し代入
 - 入力：メンバの情報
 - 出力：Member型へのポインタ
- read_members_from_file
 - 処理：CSVファイルからのメンバ情報の読み出し
 - 入力：filename(char\*):ファイル名，ret_num(int\*):データサイズへのポインタ，member_array(Member\*\*):メンバへの配列
 - 出力：情報格納後のメンバへの配列
- init_members_from_file
 - 処理：CSVファイルからの初期メンバの読み出し
- add_members_from_file
 - 処理：CSVファイルからの追加メンバの読み出し

ソート関数の呼び出し
- launch_sort
 - 処理：ソート関数の呼び出しを行う。LAUNCH_SORTマクロを使うことにより、ソート関数名、比較関数名、昇順/降順を自動的に表示する。

In [ ]:
%%file tool.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include "main.h"
void swap(Member *data[], int idx1, int idx2) {
    Member *temp;
    if (idx1 != idx2) {
        temp = data[idx1];
        data[idx1] = data[idx2];
        data[idx2] = temp;
    }
    return;
}
void printarray(int n, Member *data[]) {
    for(int i=0; i<n; i++){
        printf("%s(%s): 年齢:%d, 身長:%3.1lf, 強さ:%d, 所属:%s, 属性:%s\n",
               data[i]->name, data[i]->yomi, data[i]->age, data[i]->height,
               data[i]->score, data[i]->group, data[i]->attr);
    }
    return;
}
void copyarray(int n, Member *data1[], Member *data2[]) {
    for(int i=0; i<n; i++){
        data2[i] = data1[i];
    }
    return;
}
/**
 * カウンタの操作
 */
Counter ctr;
Counter total_ctr;
void print_counter(void) {
    printf("lap: compare = %d, swap = %d, move = %d\n", ctr.compare, ctr.swap, ctr.move);
    printf("all: compare = %d, swap = %d, move = %d\n",
           total_ctr.compare+ctr.compare, total_ctr.swap+ctr.swap, total_ctr.move+ctr.move);
}
void reset_counter(void) {
    total_ctr.compare += ctr.compare;
    total_ctr.swap    += ctr.swap;
    total_ctr.move    += ctr.move;
    ctr.compare = ctr.swap = ctr.move = 0;
}

/**
 * メンバの割り当てと読み込み
 */
Member *allocate_member(const char *name, const char *yomi, const char *group, const char *attr, int age, float height, int score) {
    Member *new_member;
    new_member = (Member *)malloc(sizeof(Member));
    if (new_member == NULL) {
        printf("Cannot allocate new member\n");
        return NULL;
    }
    strncpy(new_member->name, name, MAX_NAME);
    strncpy(new_member->yomi, yomi, MAX_NAME);
    strncpy(new_member->group, group, MAX_NAME);
    strncpy(new_member->attr, attr, MAX_ATTR);
    new_member->age = age;
    new_member->height = height;
    new_member->score = score;
    return new_member;
}
Member **read_members_from_file(const char *filename, int *ret_num, Member **member_array) {
    FILE *fin;
    char buf[BUFSIZ];
    int num_members, i;
    char name[MAX_NAME+1];
    char yomi[MAX_NAME+1];
    char group[MAX_NAME+1];
    char attr[MAX_ATTR+1];
    int age;
    float height;
    int score;

    fin = fopen(filename, "r");
    if (fin == NULL) {
        printf("Cannot open %s\n", filename);
        return NULL;
    }
    if (fgets(buf, BUFSIZ, fin) == NULL || sscanf(buf, "%d", &num_members) < 1) {
        printf("Illegal format %s\n", filename);
        fclose(fin);
        return NULL;
    }
    if (member_array == NULL) {
        member_array = (Member**)malloc(sizeof(Member) * num_members);
        if (member_array == NULL) {
            printf("Cannot allocate array\n");
            fclose(fin);
            return NULL;
        }
    } else {
        Member **tmp_member_array = (Member**)realloc(member_array, (sizeof(Member) * (num_members + *ret_num)));
        if (tmp_member_array == NULL) {
            printf("Cannot reallocate array\n");
            fclose(fin);
            return member_array;
        }
        member_array = tmp_member_array;
    }
    for (i = 0; i < num_members; i++) {
        if (fgets(buf, BUFSIZ, fin) == NULL ||
            sscanf(buf, "%[^,],%[^,],%[^,],%[^,],%d,%f,%d",
                   name, yomi, group, attr, &age, &height, &score) < 7){
            printf("Illegal format %s in line %d\n", filename, i+1);
            break;
        }
        member_array[*ret_num + i] = allocate_member(name, yomi, group ,attr, age, height, score);
    }
    fclose(fin);
    *ret_num += i;
    return member_array;
}
Member **init_members_from_file(const char *filename, int *ret_num) {
    *ret_num = 0;
    return read_members_from_file(filename, ret_num, NULL);
}
Member **add_members_from_file(const char *filename, int *ret_num, Member **member_array) {
    return read_members_from_file(filename, ret_num, member_array);
}
/**
 * ソート関数を呼び出し、その時のソート関数名、比較関数名、昇順か降順かを表示する
 */
void launch_sort(int n, Member *data[], void (*sort_func)(int, Member**, CmpFunc, int), CmpFunc cmp, int descending,
                 const char *sort_name, const char *cmp_name) {
    printf("\n%s with %s(%s)\n", sort_name, cmp_name, descending ? "descending" : "ascending");
    sort_func(n, data, cmp, descending);
}

Writing tool.c


##(1c)比較関数

In [ ]:
%%file compare.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

// 名前(読み)(yomi)で比較
int compare_by_yomi(const Member *data1, const Member *data2, int descending) {
    int result = strcmp(data1->yomi, data2->yomi);
    if (descending) {
        result = -result;
    }
    return result;
}
// 年齢(age)で比較
int compare_by_age(const Member *data1, const Member *data2, int descending) {
    int result;
    // ダミー
    return 0;
}
// 身長(height)で比較
int compare_by_height(const Member *data1, const Member *data2, int descending) {
    int result;
    // ダミー
    return 0;
}
// 強さ(score)で比較
int compare_by_score(const Member *data1, const Member *data2, int descending) {
    int result;
    // ダミー
    return result;
}
// 所属(group)で比較
int compare_by_group(const Member *data1, const Member *data2, int descending) {
    int result;

    return result;
}

Writing compare.c


##(1d)メンバ定義

最初の1行目に全部で何人分のデータが続くかを示す整数。
以下、各行に

名前,読み,所属,属性,年齢,身長,強さ

の順に与える。

In [ ]:
%%file members.csv
15
竈門炭治郎,カマドタンジロウ,蒲鉾,水,16,165.0,40
冨岡義勇,トミオカギユウ,柱,水,21,176.0,80
竈門禰豆子,カマドネズコ,蒲鉾,鬼,14,150.0,30
我妻善逸,アガツマゼンイツ,蒲鉾,雷,16,164.5,40
栗花落カナヲ,ツユリカナヲ,五感,花,16,156.0,50
嘴平伊之助,ハシビライノスケ,蒲鉾,獣,15,164.0,40
胡蝶しのぶ,コチョウシノブ,柱,蟲,18,151.0,70
煉獄杏寿郎,レンゴクキョウジュロウ,柱,炎,20,177.0,80
宇随天元,ウズイテンゲン,柱,音,23,198.0,80
時透無一郎,トキトウムイチロウ,柱,霞,14,160.0,90
甘露寺蜜璃,カンロジミツリ,柱,恋,19,167.0,70
伊黒小芭内,イグロオバナイ,柱,蛇,21,162.0,70
不死川玄弥,シナズガワゲンヤ,五感,鬼,16,180.0,30
不死川実弥,シナズガワサネミ,柱,風,21,179.0,90
悲鳴嶋行冥,ヒメジマギョウメイ,柱,岩,27,220.0,100


Writing members.csv


In [ ]:
%%file members2.csv
2
鱗滝左近次,ウロコダキサコンジ,育手,水,55,170.5,60
桑島慈悟郎,クワジマジゴロウ,育手,雷,60,165.5,50

Writing members2.csv


#（２）ソートプログラム群

- ソートプログラム群の共通形式
 - 入力 n(int)：データサイズ，data\[\](Member\*)：配列データ，cmp：比較関数，descending：0なら昇順・0以外なら降順
 - 出力 data\[\](Member\*)：ソートされた配列データ


##(2a)バブルソート オリジナル

In [ ]:
%%file bubble_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void bubble_sort(int n, Member *data[], CmpFunc cmp, int descending){
    reset_counter();
    for (int i=n-1; i>=1; i--) {
        for (int j=1; j<=i; j++) {
            ctr.compare++;
            if (cmp(data[j-1], data[j], descending)> 0) {
                swap(data, j-1, j);
                ctr.swap++;
            }
        }
    }
    return;
}

Writing bubble_sort.c


##(2b)バブルソート改良版

In [ ]:
%%file bubble2_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void bubble2_sort(int n, Member *data[], CmpFunc cmp, int descending){
    int jswap, jmax;
    reset_counter();
    jmax = n-1;
    for (int i=n-1; i>=1; i--) {
            jswap = 1;
        for (int j=1; j<=jmax; j++) {
            ctr.compare++;
            if (cmp(data[j-1], data[j], descending) > 0) {
                jswap = j;
                swap(data, j-1, j);
                ctr.swap++;
            }
        }
        jmax = jswap-1;
    }
    return;
}

Writing bubble2_sort.c


##(2c)シェーカーソート

In [ ]:
%%file shaker_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void shaker_sort(int n, Member *data[], CmpFunc cmp, int descending){
    int jmin=0, jmax=n-1, j, temp, jswap;
    reset_counter();
    while(1){
        if (jmin>=jmax) break;
        jswap = jmax;
        for (j=jmax; j>jmin; j--) {
            ctr.compare++;
            if (cmp(data[j-1], data[j], descending) > 0) {
                jswap = j;
                swap(data, j-1, j);
                ctr.swap++;
            }
        }
        jmin = jswap;
        if (jmin>=jmax) break;
        jswap = 0;
        for (j=jmin; j<jmax; j++) {
            ctr.compare++;
            if (cmp(data[j], data[j+1], descending) > 0) {
                jswap = j;
                swap(data, j, j+1);
                ctr.swap++;
            }
        }
        jmax = jswap;
    }
}

Writing shaker_sort.c


##(2d)選択ソート

In [ ]:
%%file select_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void select_sort(int n, Member *data[], CmpFunc cmp, int descending){
    int i_min;
    Member *min;
    reset_counter();
    for (int i=0; i<n-1; i++) {
        i_min=i;
        min=data[i];
        for (int j=i+1; j<n; j++) {
            if (cmp(min, data[j], descending) > 0){
                i_min=j;
                min=data[j];
            }
            ctr.compare++;
        }
        swap(data, i_min, i);
        ctr.swap++;
    }
    return;
}

Writing select_sort.c


##(2e)挿入ソート

In [ ]:
%%file insert_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void insert_sort(int n, Member *data[], CmpFunc cmp, int descending){
    reset_counter();
    for (int i=1; i<n; i++) {
        for (int j=i; j>0; j--) {
            ctr.compare++;
            if (cmp(data[j-1], data[j], descending) > 0) {
                swap(data, j-1, j);
                ctr.swap++;
            }
            else break;
        }
    }
    return;
}

Writing insert_sort.c


##(2f)クイックソート

In [ ]:
%%file quick_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void quicksort(Member *data[], CmpFunc cmp, int descending, int l, int r){
    int mid, i, j;
    Member *pivot;
    mid=(l+r)/2; pivot=data[mid];
    i=l; j=r;
    while (i<=j) {
        ctr.compare++;
        while (cmp(data[i], pivot, descending) < 0) {
            i=i+1;
            ctr.compare++;
        }
        ctr.compare++;
        while (cmp(pivot, data[j], descending) < 0) {
            j=j-1;
            ctr.compare++;
        }
        if(i<=j){
            swap(data, i, j);
            ctr.swap++;
            i=i+1; j=j-1;
        }
    }
    if (l<j)quicksort(data, cmp, descending, l, j);
    if (i<r)quicksort(data, cmp, descending, i, r);
    return;
}
void quick_sort(int n, Member *data[], CmpFunc cmp, int descending){
    reset_counter();
    quicksort(data, cmp, descending, 0, n-1);
    return;
}

Writing quick_sort.c


##(2g)コンパイル

In [ ]:
!gcc tool.c -c
!gcc compare.c -c
!gcc bubble_sort.c -c
!gcc bubble2_sort.c -c
!gcc shaker_sort.c -c
!gcc select_sort.c -c
!gcc insert_sort.c -c
!gcc quick_sort.c -c

#演習テスト 課題１ 比較関数

##課題1 比較関数
1. 年齢(age)で比較する関数 compare_by_age を作成せよ。
2. 身長(height)で比較する関数 compare_by_height を作成せよ。
3. 強さ(score)で比較する関数 compare_by_score を作成せよ。
4. 所属(group)で比較する関数 compare_by_group を作成せよ。

なお、比較関数は data1, data2, descending を引数として取り、
+ data1 が data2 より前にあるべき時は -1
+ data1 が data2 と同じ時は 0
+ data1 が data2 より後ろにあるべき時は 1

を返す関数とする。descending が 0以外の場合は、符号が逆転する。

In [ ]:
%%file compare.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

// 名前(読み)(yomi)で比較
int compare_by_yomi(const Member *data1, const Member *data2, int descending) {
    int result = strcmp(data1->yomi, data2->yomi);
    if (descending) {
        result = -result;
    }
    return result;
}
// 課題1-1: 年齢(age)で比較
int compare_by_age(const Member *data1, const Member *data2, int descending) {
    int result;
    // ここを記述する
    return result;
}
// 課題1-2: 身長(height)で比較
int compare_by_height(const Member *data1, const Member *data2, int descending) {
    int result;
    // ここを記述する
    return result;
}
// 課題1-3: 強さ(score)で比較
int compare_by_score(const Member *data1, const Member *data2, int descending) {
    int result;
    // ここを記述する
    return result;
}
// 課題1-4: 所属(group)で比較
int compare_by_group(const Member *data1, const Member *data2, int descending) {
    int result;
    // ここを記述する
    return result;
}

Overwriting compare.c


###メインプログラム

In [ ]:
%%file test1.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

int main(void) {
    Member **member_array;
    int num_members;

    // メンバーの読み込み
    member_array = init_members_from_file("members.csv", &num_members);

    printf("initial:\n");
    printarray(num_members, member_array);

    // 名前の読み(yomi)順にソート
    LAUNCH_SORT(num_members, member_array, quick_sort, compare_by_yomi, 0);
    printarray(num_members, member_array);
    print_counter();

    // 課題1-1: 身長(height)の高い順にソート
    printf("\n課題1-1: ");
    LAUNCH_SORT(num_members, member_array, shaker_sort, compare_by_height, 1);
    printarray(num_members, member_array);
    print_counter();

    // 課題1-2: 年齢(age)の若い順にソート
    printf("\n課題1-2: ");
    LAUNCH_SORT(num_members, member_array, insert_sort, compare_by_age, 0);
    printarray(num_members, member_array);
    print_counter();

    // 課題1-3: 強さ(score)の強い順にソート
    printf("\n課題1-3: ");
    LAUNCH_SORT(num_members, member_array, select_sort, compare_by_score, 1);
    printarray(num_members, member_array);
    print_counter();

    // 課題1-4: 所属(group)ごとにソート
    printf("\n課題1-4: ");
    LAUNCH_SORT(num_members, member_array, bubble_sort, compare_by_group, 0);
    printarray(num_members, member_array);
    print_counter();

    return 0;
}

Writing test1.c


###コンパイル（test1.c）＆リンク

In [ ]:
!gcc compare.c -c
!gcc test1.c -c
!gcc tool.o compare.o bubble_sort.o bubble2_sort.o shaker_sort.o insert_sort.o select_sort.o quick_sort.o test1.o -o test1

###実行

In [ ]:
!./test1

initial:
竈門炭治郎(カマドタンジロウ): 年齢:16, 身長:165.0, 強さ:40, 所属:蒲鉾, 属性:水
冨岡義勇(トミオカギユウ): 年齢:21, 身長:176.0, 強さ:80, 所属:柱, 属性:水
竈門禰豆子(カマドネズコ): 年齢:14, 身長:150.0, 強さ:30, 所属:蒲鉾, 属性:鬼
我妻善逸(アガツマゼンイツ): 年齢:16, 身長:164.5, 強さ:40, 所属:蒲鉾, 属性:雷
栗花落カナヲ(ツユリカナヲ): 年齢:16, 身長:156.0, 強さ:50, 所属:五感, 属性:花
嘴平伊之助(ハシビライノスケ): 年齢:15, 身長:164.0, 強さ:40, 所属:蒲鉾, 属性:獣
胡蝶しのぶ(コチョウシノブ): 年齢:18, 身長:151.0, 強さ:70, 所属:柱, 属性:蟲
煉獄杏寿郎(レンゴクキョウジュロウ): 年齢:20, 身長:177.0, 強さ:80, 所属:柱, 属性:炎
宇随天元(ウズイテンゲン): 年齢:23, 身長:198.0, 強さ:80, 所属:柱, 属性:音
時透無一郎(トキトウムイチロウ): 年齢:14, 身長:160.0, 強さ:90, 所属:柱, 属性:霞
甘露寺蜜璃(カンロジミツリ): 年齢:19, 身長:167.0, 強さ:70, 所属:柱, 属性:恋
伊黒小芭内(イグロオバナイ): 年齢:21, 身長:162.0, 強さ:70, 所属:柱, 属性:蛇
不死川玄弥(シナズガワゲンヤ): 年齢:16, 身長:180.0, 強さ:30, 所属:五感, 属性:鬼
不死川実弥(シナズガワサネミ): 年齢:21, 身長:179.0, 強さ:90, 所属:柱, 属性:風
悲鳴嶋行冥(ヒメジマギョウメイ): 年齢:27, 身長:220.0, 強さ:100, 所属:柱, 属性:岩

quick_sort with compare_by_yomi(ascending)
我妻善逸(アガツマゼンイツ): 年齢:16, 身長:164.5, 強さ:40, 所属:蒲鉾, 属性:雷
伊黒小芭内(イグロオバナイ): 年齢:21, 身長:162.0, 強さ:70, 所属:柱, 属性:蛇
宇随天元(ウズイテンゲン): 年齢:23, 身長:198.0, 強さ:80, 所属:柱, 属性:音
竈門炭治郎(カマドタンジロ

#演習テスト 課題2 追加されたメンバに対するソート

##課題2-1 名前(読み)順にソート
1. メンバを読み込み、名前(読み)(yomi)順にソートした後に、追加メンバを読み込み、再び全体を名前(読み)(yomi)順にソートせよ。
2. この時、トータルの比較(compare)回数が最も少なくなる組合せはどれとどれか。
3. この時、トータルの交換(swap)回数が最も少なくなる組合せはどれとどれか。

###メインプログラム

In [ ]:
%%file test2-1.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

int main(void) {
    Member **member_array;
    int num_members;

    member_array = init_members_from_file("members.csv", &num_members);

    printf("initial:\n");
    printarray(num_members, member_array);

    LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
    printarray(num_members, member_array);
    print_counter();

    member_array = add_members_from_file("members2.csv", &num_members, member_array);
    printf("\nadded:\n");
    printarray(num_members, member_array);

    LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
    printarray(num_members, member_array);
    print_counter();

    return 0;
}

Writing test2-1.c


###コンパイル(test2-1.c)＆リンク

In [ ]:
!gcc test2-1.c -c
!gcc tool.o compare.o bubble_sort.o bubble2_sort.o shaker_sort.o insert_sort.o select_sort.o quick_sort.o test2-1.o -o test2-1

test2-1.c: In function ‘main’:
test2-1.c:15:71: warning: trigraph ??) ignored, use -trigraphs to enable []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wtrigraphs-Wtrigraphs]8;;]
   15 |     LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
      |                                                                        
In file included from test2-1.c:4:
test2-1.c:15:44: error: expected expression before ‘?’ token
   15 |     LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
      |                                            ^
main.h:44:80: note: in definition of macro ‘LAUNCH_SORT’
   44 | #define LAUNCH_SORT(n, data, sort_func, cmp, descending)  launch_sort(n, data, sort_func, cmp, descending, #sort_func, #cmp)
      |                                                                                ^~~~~~~~~
test2-1.c:15:47: error: ‘_sort’ undeclared (first use in this function); did you mean ‘qsort’?
   15 |     LAUNCH_SO

###実行

In [ ]:
!./test2-1

/bin/bash: line 1: ./test2-1: No such file or directory


##課題2-2 年齢の高い順にソート
1. メンバを読み込み、年齢(age)の高い順にソートした後に、追加メンバを読み込み、再び全体を年齢(age)の高い順にソートせよ。
2. この時、トータルの比較(compare)回数が最も少なくなる組合せはどれとどれか。
3. この時、トータルの交換(swap)回数が最も少なくなる組合せはどれとどれか。

###メインプログラム

In [ ]:
%%file test2-2.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

int main(void) {
    Member **member_array;
    int num_members;

    member_array = init_members_from_file("members.csv", &num_members);

    printf("initial:\n");
    printarray(num_members, member_array);

    LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
    printarray(num_members, member_array);
    print_counter();

    member_array = add_members_from_file("members2.csv", &num_members, member_array);
    printf("\nadded:\n");
    printarray(num_members, member_array);

    LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
    printarray(num_members, member_array);
    print_counter();

    return 0;
}

Writing test2-2.c


###コンパイル(test2-2.c)＆リンク

In [ ]:
!gcc test2-2.c -c
!gcc tool.o compare.o bubble_sort.o bubble2_sort.o shaker_sort.o insert_sort.o select_sort.o quick_sort.o test2-2.o -o test2-2

test2-2.c: In function ‘main’:
test2-2.c:15:71: warning: trigraph ??) ignored, use -trigraphs to enable []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wtrigraphs-Wtrigraphs]8;;]
   15 |     LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
      |                                                                        
In file included from test2-2.c:4:
test2-2.c:15:44: error: expected expression before ‘?’ token
   15 |     LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
      |                                            ^
main.h:44:80: note: in definition of macro ‘LAUNCH_SORT’
   44 | #define LAUNCH_SORT(n, data, sort_func, cmp, descending)  launch_sort(n, data, sort_func, cmp, descending, #sort_func, #cmp)
      |                                                                                ^~~~~~~~~
test2-2.c:15:47: error: ‘_sort’ undeclared (first use in this function); did you mean ‘qsort’?
   15 |     LAUNCH_SO

###実行

In [ ]:
!./test2-2

/bin/bash: line 1: ./test2-2: No such file or directory


#発展課題 安定したソート

##発展課題1
1. メンバを読み込み、所属(group)の順にソートせよ。ただし、同じ所属のメンバの中では名前(読み)(yomi)順にソートされているようにせよ。
2. この時、最も比較(compare)回数が少なくなる組合せは、どれとどれか。
3. この時、最も交換(swap)回数が少なくなる組合せは、どれとどれか。

###メインプログラム

In [ ]:
%%file test3-1.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

int main(void) {
    Member **member_array;
    int num_members;

    member_array = init_members_from_file("members.csv", &num_members);

    printf("initial:\n");
    printarray(num_members, member_array);

    LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
    printarray(num_members, member_array);
    print_counter();

    LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
    printarray(num_members, member_array);
    print_counter();

    return 0;
}

Writing test3-1.c


###コンパイル(test3-1.c)＆リンク

In [ ]:
!gcc test3-1.c -c
!gcc tool.o compare.o bubble_sort.o bubble2_sort.o shaker_sort.o insert_sort.o select_sort.o quick_sort.o test3-1.o -o test3-1

test3-1.c: In function ‘main’:
test3-1.c:15:71: warning: trigraph ??) ignored, use -trigraphs to enable []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wtrigraphs-Wtrigraphs]8;;]
   15 |     LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
      |                                                                        
In file included from test3-1.c:4:
test3-1.c:15:44: error: expected expression before ‘?’ token
   15 |     LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
      |                                            ^
main.h:44:80: note: in definition of macro ‘LAUNCH_SORT’
   44 | #define LAUNCH_SORT(n, data, sort_func, cmp, descending)  launch_sort(n, data, sort_func, cmp, descending, #sort_func, #cmp)
      |                                                                                ^~~~~~~~~
test3-1.c:15:47: error: ‘_sort’ undeclared (first use in this function); did you mean ‘qsort’?
   15 |     LAUNCH_SO

###実行

In [ ]:
!./test3-1

/bin/bash: line 1: ./test3-1: No such file or directory


##発展課題2
1. メンバを読み込み、強さ(score)の強い順にソートせよ。ただし、同じ強さの場合は、年齢(age)の高い方から順に並ぶようにせよ。
2. この時、比較(compare)回数が最も少なくなる組合せは、どれとどれか。
3. この時、交換(swap)回数が最も少なくなる組合せは、どれとどれか。

###メインプログラム

In [ ]:
%%file test3-2.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

int main(void) {
    Member **member_array;
    int num_members;

    member_array = init_members_from_file("members.csv", &num_members);

    printf("initial:\n");
    printarray(num_members, member_array);

    LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
    printarray(num_members, member_array);
    print_counter();

    LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
    printarray(num_members, member_array);
    print_counter();

    return 0;
}

Writing test3-2.c


###コンパイル(test3-2.c)＆リンク

In [ ]:
!gcc test3-2.c -c
!gcc tool.o compare.o bubble_sort.o bubble2_sort.o shaker_sort.o insert_sort.o select_sort.o quick_sort.o test3-2.o -o test3-2

test3-2.c: In function ‘main’:
test3-2.c:15:71: warning: trigraph ??) ignored, use -trigraphs to enable []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wtrigraphs-Wtrigraphs]8;;]
   15 |     LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
      |                                                                        
In file included from test3-2.c:4:
test3-2.c:15:44: error: expected expression before ‘?’ token
   15 |     LAUNCH_SORT(num_members, member_array, ???_sort, compare_by_???, ???);
      |                                            ^
main.h:44:80: note: in definition of macro ‘LAUNCH_SORT’
   44 | #define LAUNCH_SORT(n, data, sort_func, cmp, descending)  launch_sort(n, data, sort_func, cmp, descending, #sort_func, #cmp)
      |                                                                                ^~~~~~~~~
test3-2.c:15:47: error: ‘_sort’ undeclared (first use in this function); did you mean ‘qsort’?
   15 |     LAUNCH_SO

###実行

In [ ]:
!./test3-2

/bin/bash: line 1: ./test3-2: No such file or directory
